In [120]:
import urllib
import pandas as pd
import collections
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
%matplotlib inline

In [4]:
facts = pd.read_table('cases_55-15.csv',sep='|')

In [5]:
facts.head()

,year,docket_number,facts_of_the_case,question
0,1956,442,"The Food and Drug Administration (""FDA"") fired...",When may an employee be discharged under the V...
1,1956,10,"Nelson, a member of the Communist Party, was c...",Did the Smith Act supersede enforcement of Pen...
2,1956,95,Judson Griffin and James Crenshaw were indicte...,Did requiring Griffin and Crenshaw to pay a fe...
3,1957,701,While residing at an airbase in England as a m...,Do American citizens abroad retain the rights ...
4,1958,70,"In 1944, United States Army private Albert Tro...",Did Section 401(g) of the amended 1940 Nationa...


In [7]:
df_justice_all = pd.read_csv('SCDB_2015_02_justiceCentered_LegalProvision.csv', encoding="ISO-8859-1")

In [14]:
df_justice_all[90000:90010]

,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,sctCite,ledCite,lexisCite,...,majVotes,minVotes,justice,justiceName,vote,opinion,direction,majority,firstAgreement,secondAgreement
90000,1994-004,1994-004-01,1994-004-01-01,1994-004-01-01-01-04,11/8/1994,1,513 U.S. 18,115 S. Ct. 386,130 L. Ed. 2d 233,1994 U.S. LEXIS 7982,...,9,0,105,AScalia,1,2,2,2,NaN,NaN
90001,1994-004,1994-004-01,1994-004-01-01,1994-004-01-01-01-05,11/8/1994,1,513 U.S. 18,115 S. Ct. 386,130 L. Ed. 2d 233,1994 U.S. LEXIS 7982,...,9,0,106,AMKennedy,1,1,2,2,NaN,NaN
90002,1994-004,1994-004-01,1994-004-01-01,1994-004-01-01-01-06,11/8/1994,1,513 U.S. 18,115 S. Ct. 386,130 L. Ed. 2d 233,1994 U.S. LEXIS 7982,...,9,0,107,DHSouter,1,1,2,2,NaN,NaN
90003,1994-004,1994-004-01,1994-004-01-01,1994-004-01-01-01-07,11/8/1994,1,513 U.S. 18,115 S. Ct. 386,130 L. Ed. 2d 233,1994 U.S. LEXIS 7982,...,9,0,108,CThomas,1,1,2,2,NaN,NaN
90004,1994-004,1994-004-01,1994-004-01-01,1994-004-01-01-01-08,11/8/1994,1,513 U.S. 18,115 S. Ct. 386,130 L. Ed. 2d 233,1994 U.S. LEXIS 7982,...,9,0,109,RBGinsburg,1,1,2,2,NaN,NaN
90005,1994-004,1994-004-01,1994-004-01-01,1994-004-01-01-01-09,11/8/1994,1,513 U.S. 18,115 S. Ct. 386,130 L. Ed. 2d 233,1994 U.S. LEXIS 7982,...,9,0,110,SGBreyer,1,1,2,2,NaN,NaN
90006,1994-005,1994-005-01,1994-005-01-01,1994-005-01-01-01-01,11/14/1994,1,513 U.S. 30,115 S. Ct. 394,130 L. Ed. 2d 245,1994 U.S. LEXIS 7983,...,5,4,102,WHRehnquist,2,1,2,1,104,NaN
90007,1994-005,1994-005-01,1994-005-01-01,1994-005-01-01-01-02,11/14/1994,1,513 U.S. 30,115 S. Ct. 394,130 L. Ed. 2d 245,1994 U.S. LEXIS 7983,...,5,4,103,JPStevens,3,2,1,2,NaN,NaN
90008,1994-005,1994-005-01,1994-005-01-01,1994-005-01-01-01-03,11/14/1994,1,513 U.S. 30,115 S. Ct. 394,130 L. Ed. 2d 245,1994 U.S. LEXIS 7983,...,5,4,104,SDOConnor,2,2,2,1,NaN,NaN
90009,1994-005,1994-005-01,1994-005-01-01,1994-005-01-01-01-04,11/14/1994,1,513 U.S. 30,115 S. Ct. 394,130 L. Ed. 2d 245,1994 U.S. LEXIS 7983,...,5,4,105,AScalia,2,1,2,1,104,NaN


In [16]:
df_justice_all.columns

Index(['caseId', 'docketId', 'caseIssuesId', 'voteId', 'dateDecision',
       'decisionType', 'usCite', 'sctCite', 'ledCite', 'lexisCite', 'term',
       'naturalCourt', 'chief', 'docket', 'caseName', 'dateArgument',
       'dateRearg', 'petitioner', 'petitionerState', 'respondent',
       'respondentState', 'jurisdiction', 'adminAction', 'adminActionState',
       'threeJudgeFdc', 'caseOrigin', 'caseOriginState', 'caseSource',
       'caseSourceState', 'lcDisagreement', 'certReason', 'lcDisposition',
       'lcDispositionDirection', 'declarationUncon', 'caseDisposition',
       'caseDispositionUnusual', 'partyWinning', 'precedentAlteration',
       'voteUnclear', 'issue', 'issueArea', 'decisionDirection',
       'decisionDirectionDissent', 'authorityDecision1', 'authorityDecision2',
       'lawType', 'lawSupp', 'lawMinor', 'majOpinWriter', 'majOpinAssigner',
       'splitVote', 'majVotes', 'minVotes', 'justice', 'justiceName', 'vote',
       'opinion', 'direction', 'majority', 'firstA

In [17]:
judges = ['AScalia', 'AMKennedy', 'CThomas','RBGinsburg', 'SGBreyer', 'JGRoberts', 'SAAlito', 'SSotomayor','EKagan']

In [21]:
df_justice_all['justiceName'].tail(25)

110048     JGRoberts
110049       AScalia
110050     AMKennedy
110051       CThomas
110052    RBGinsburg
110053      SGBreyer
110054    SSotomayor
110055        EKagan
110056     JGRoberts
110057       AScalia
110058     AMKennedy
110059       CThomas
110060    RBGinsburg
110061      SGBreyer
110062       SAAlito
110063    SSotomayor
110064        EKagan
110065     JGRoberts
110066       AScalia
110067     AMKennedy
110068       CThomas
110069      SGBreyer
110070       SAAlito
110071    SSotomayor
110072        EKagan
Name: justiceName, dtype: object

In [23]:
df = pd.merge(df_justice_all,facts,left_on='docket',right_on='docket_number')

In [24]:
df.tail()

,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,sctCite,ledCite,lexisCite,...,vote,opinion,direction,majority,firstAgreement,secondAgreement,year,docket_number,facts_of_the_case,question
46863,2014-075,2014-075-01,2014-075-01-01,2014-075-01-01-01-04,6/1/2015,2,NaN,NaN,NaN,2015 U.S. LEXIS 3715,...,1,1,1,2,NaN,NaN,2015,14-939,"On November 13, 2004, Christopher Barkes was a...",Is there a clearly established constitutional ...
46864,2014-075,2014-075-01,2014-075-01-01,2014-075-01-01-01-06,6/1/2015,2,NaN,NaN,NaN,2015 U.S. LEXIS 3715,...,1,1,1,2,NaN,NaN,2015,14-939,"On November 13, 2004, Christopher Barkes was a...",Is there a clearly established constitutional ...
46865,2014-075,2014-075-01,2014-075-01-01,2014-075-01-01-01-07,6/1/2015,2,NaN,NaN,NaN,2015 U.S. LEXIS 3715,...,1,1,1,2,NaN,NaN,2015,14-939,"On November 13, 2004, Christopher Barkes was a...",Is there a clearly established constitutional ...
46866,2014-075,2014-075-01,2014-075-01-01,2014-075-01-01-01-08,6/1/2015,2,NaN,NaN,NaN,2015 U.S. LEXIS 3715,...,1,1,1,2,NaN,NaN,2015,14-939,"On November 13, 2004, Christopher Barkes was a...",Is there a clearly established constitutional ...
46867,2014-075,2014-075-01,2014-075-01-01,2014-075-01-01-01-09,6/1/2015,2,NaN,NaN,NaN,2015 U.S. LEXIS 3715,...,1,1,1,2,NaN,NaN,2015,14-939,"On November 13, 2004, Christopher Barkes was a...",Is there a clearly established constitutional ...


In [25]:
cur_df = df[df['justiceName'].isin(judges)] #DF with only the current judges + Scalia

In [27]:
print (len(df))
print (len(cur_df))

46868
12362


In [57]:
def tfvecto(list_of_facts):
    tfvec = TfidfVectorizer(analyzer='word', stop_words=stopwords.words('english'))
    X = tfvec.fit_transform(list_of_facts)
    return X

In [30]:
test = facts.facts_of_the_case.tail(30)

In [32]:
vec = CountVectorizer(analyzer='word', stop_words=stopwords.words('english') )

In [33]:
X = vec.fit_transform(test)

In [44]:
X[1].toarray()

array([[0, 0, 0, ..., 1, 0, 0]])

In [36]:
tfvec = TfidfVectorizer(analyzer='word', stop_words=stopwords.words('english'))

In [37]:
XX = tfvec.fit_transform(test)

In [45]:
XX[1].toarray()

array([[ 0.        ,  0.        ,  0.        , ...,  0.08988443,
         0.        ,  0.        ]])

In [46]:
tfvec.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': ['i',
  'me',
  'my',
  'myself',
  'we',
  'our',
  'ours',
  'ourselves',
  'you',
  'your',
  'yours',
  'yourself',
  'yourselves',
  'he',
  'him',
  'his',
  'himself',
  'she',
  'her',
  'hers',
  'herself',
  'it',
  'its',
  'itself',
  'they',
  'them',
  'their',
  'theirs',
  'themselves',
  'what',
  'which',
  'who',
  'whom',
  'this',
  'that',
  'these',
  'those',
  'am',
  'is',
  'are',
  'was',
  'were',
  'be',
  'been',
  'being',
  'have',
  'has',
  'had',
  'having',
  'do',
  'does',
  'did',
  'doing',
  'a',
  'an',
  'the',
  'and',
  'but',
  'if',
  'or',
  'because',
  'as',
  'until',
  'while',
  'of',
  'at',
  'by',
  'for',
  'with',
  'about',
  'ag

In [49]:
len(cur_df[['justice', 'justiceName']])

12362

In [95]:
cur_df2 = cur_df[cur_df.direction.notnull()]

In [99]:
cur_df2.reset_index()

,index,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,sctCite,ledCite,...,vote,opinion,direction,majority,firstAgreement,secondAgreement,year,docket_number,facts_of_the_case,question
0,23189,1999-080,1999-080-01,1999-080-01-01,1999-080-01-01-01-04,6/19/2000,1,530 U.S. 392,120 S. Ct. 2304,147 L. Ed. 2d 374,...,1,1,2,2,NaN,NaN,2000,8 ORIG,"In 1952, Arizona invoked the U.S. Supreme Cour...",Are the Quechan Tribe and the United States cl...
1,23190,1999-080,1999-080-01,1999-080-01-01,1999-080-01-01-01-04,6/19/2000,1,530 U.S. 392,120 S. Ct. 2304,147 L. Ed. 2d 374,...,1,1,2,2,NaN,NaN,2000,8 ORIG,"In 1952, Arizona invoked the U.S. Supreme Cour...",Are the Quechan Tribe and the United States cl...
2,23191,1999-080,1999-080-01,1999-080-01-01,1999-080-01-01-01-05,6/19/2000,1,530 U.S. 392,120 S. Ct. 2304,147 L. Ed. 2d 374,...,1,1,2,2,NaN,NaN,2000,8 ORIG,"In 1952, Arizona invoked the U.S. Supreme Cour...",Are the Quechan Tribe and the United States cl...
3,23192,1999-080,1999-080-01,1999-080-01-01,1999-080-01-01-01-05,6/19/2000,1,530 U.S. 392,120 S. Ct. 2304,147 L. Ed. 2d 374,...,1,1,2,2,NaN,NaN,2000,8 ORIG,"In 1952, Arizona invoked the U.S. Supreme Cour...",Are the Quechan Tribe and the United States cl...
4,23195,1999-080,1999-080-01,1999-080-01-01,1999-080-01-01-01-07,6/19/2000,1,530 U.S. 392,120 S. Ct. 2304,147 L. Ed. 2d 374,...,2,1,1,1,102,NaN,2000,8 ORIG,"In 1952, Arizona invoked the U.S. Supreme Cour...",Are the Quechan Tribe and the United States cl...
5,23196,1999-080,1999-080-01,1999-080-01-01,1999-080-01-01-01-07,6/19/2000,1,530 U.S. 392,120 S. Ct. 2304,147 L. Ed. 2d 374,...,2,1,1,1,102,NaN,2000,8 ORIG,"In 1952, Arizona invoked the U.S. Supreme Cour...",Are the Quechan Tribe and the United States cl...
6,23197,1999-080,1999-080-01,1999-080-01-01,1999-080-01-01-01-08,6/19/2000,1,530 U.S. 392,120 S. Ct. 2304,147 L. Ed. 2d 374,...,1,2,2,2,NaN,NaN,2000,8 ORIG,"In 1952, Arizona invoked the U.S. Supreme Cour...",Are the Quechan Tribe and the United States cl...
7,23198,1999-080,1999-080-01,1999-080-01-01,1999-080-01-01-01-08,6/19/2000,1,530 U.S. 392,120 S. Ct. 2304,147 L. Ed. 2d 374,...,1,2,2,2,NaN,NaN,2000,8 ORIG,"In 1952, Arizona invoked the U.S. Supreme Cour...",Are the Quechan Tribe and the United States cl...
8,23199,1999-080,1999-080-01,1999-080-01-01,1999-080-01-01-01-09,6/19/2000,1,530 U.S. 392,120 S. Ct. 2304,147 L. Ed. 2d 374,...,1,1,2,2,NaN,NaN,2000,8 ORIG,"In 1952, Arizona invoked the U.S. Supreme Cour...",Are the Quechan Tribe and the United States cl...
9,23200,1999-080,1999-080-01,1999-080-01-01,1999-080-01-01-01-09,6/19/2000,1,530 U.S. 392,120 S. Ct. 2304,147 L. Ed. 2d 374,...,1,1,2,2,NaN,NaN,2000,8 ORIG,"In 1952, Arizona invoked the U.S. Supreme Cour...",Are the Quechan Tribe and the United States cl...


In [50]:
dum = pd.get_dummies(cur_df['justiceName'])

In [54]:
def join_judge_facts(judges, facts_pro):
    dum = pd.get_dummies(cur_df['justiceName'])
    

,AMKennedy,AScalia,CThomas,EKagan,JGRoberts,RBGinsburg,SAAlito,SGBreyer,SSotomayor
23189,0,1,0,0,0,0,0,0,0
23190,0,1,0,0,0,0,0,0,0
23191,1,0,0,0,0,0,0,0,0
23192,1,0,0,0,0,0,0,0,0
23195,0,0,1,0,0,0,0,0,0


In [58]:
facts_processed = tfvecto(cur_df.facts_of_the_case)

In [61]:
facts_processed

<12362x14773 sparse matrix of type '<class 'numpy.float64'>'
	with 919345 stored elements in Compressed Sparse Row format>

In [ ]:
kf = cross_validation.KFold(len(cur_df), n_folds=5,shuffle = True)

In [88]:
cur_df.direction.unique()

array([  2.,   1.,  nan])

In [67]:
model_RF = RandomForestClassifier(n_estimators=80)

In [87]:
model_RF.fit(facts_processed, cur_df.direction.fillna(0).astype(int))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [71]:
cur_df.direction.shape

(12362,)

In [73]:
type(facts_processed)

scipy.sparse.csr.csr_matrix

In [74]:
import numpy as np

In [82]:
direction = np.array([cur_df.direction.T]).astype(int)

In [84]:
direction.shape

(1, 12362)

In [100]:
facts_processed[[1, 2, 5, 88]]

<4x14773 sparse matrix of type '<class 'numpy.float64'>'
	with 294 stored elements in Compressed Sparse Row format>

In [145]:
train_cols = ['issue','issueArea','lcDispositionDirection','petitioner','respondent']

In [149]:
facts_processed_array = facts_processed.toarray()

In [150]:
type(facts_processed_array)

numpy.ndarray

In [153]:
facts_df = pd.DataFrame(facts_processed_array)

In [154]:
matrix_df = pd.concat([cur_df[train_cols], facts_df], axis=1)

In [156]:
matrix_df.shape

(12362, 14778)

In [111]:
def read():
    facts = pd.read_table('cases_55-15.csv',sep='|')
    df_justice_all = pd.read_csv('SCDB_2015_02_justiceCentered_LegalProvision.csv', encoding="ISO-8859-1")
    return df_justice_all, facts

In [105]:
def tfvecto(list_of_facts):
    tfvec = TfidfVectorizer(analyzer='word', stop_words=stopwords.words('english'))
    X = tfvec.fit_transform(list_of_facts)
    return X

In [106]:
def run():
    df_justice_all, facts = read()
    df = pd.merge(df_justice_all,facts,left_on='docket',right_on='docket_number')
    cur_df = df[df['justiceName'].isin(judges)] #DF with only the current judges + Scalia
    cur_df = cur_df[cur_df.direction.notnull()] #DF where each Judge has a direction.
                                                # Can't train otherwise
    cur_df.reset_index(inplace=True)

    X = tfvecto(cur_df.facts_of_the_case)
    cross_v(X, cur_df)

In [107]:
def cross_v(X, cur_df):

    cur_df['predicted'] = 0
    kf = cross_validation.KFold(len(cur_df), n_folds=5, shuffle = True)
    scores = 0
    for train, test in kf:
        mean = 0
        score, pred = model(X, cur_df, train, test)
        # if scores:
        #     for i in range(len(score)):
        #         scores[i] += score[i]
        # else:
        #     scores = score
        scores += score
        cur_df.predicted[test] = pred
    # for score in socores:
    #     s = score/5
    #     print ('Score of CLF = ', s)
    scores = scores/5
    print ('Score of CLF = ', scores)

In [122]:
def model (X, cur_df, train, test):

    model_RF = RandomForestClassifier(n_jobs=-2, n_estimators=80)
    #model_KN = KNeighborsClassifier(n_jobs=-2, n_neighbors=10)
    #model_SVC = LinearSVC(n_jobs=-2)
    #model_Logistic = LogisticRegression(n_jobs=-2)

    #modelos = [model_RF, model_SVC, model_KN, model_Logistic]
    modelo = model_RF
    #scores = []
    #for modelo in modelos:
    print ('Fitting model ', modelo)
    modelo.fit(X[train], cur_df.direction[train].astype(int))
    print ('Predicting')
    y = modelo.predict(X[test])
    score = modelo.score(y, cur_df.direction[test])
    print ('Score: ', score)
    print ('---------------')
    #scores.append(score)
    return score, y

In [112]:
df_justice_all, facts = read()

In [113]:
df = pd.merge(df_justice_all,facts,left_on='docket',right_on='docket_number')

In [114]:
cur_df = df[df['justiceName'].isin(judges)]

In [115]:
cur_df = cur_df[cur_df.direction.notnull()]

In [117]:
cur_df.reset_index(inplace=True)

In [118]:
X = tfvecto(cur_df.facts_of_the_case)

In [127]:
kf = cross_validation.KFold(len(cur_df), n_folds=5, shuffle = True)

In [128]:
scores = 0

In [129]:
model_RF = RandomForestClassifier(n_jobs=-2, n_estimators=80)

In [130]:
train = []
test = []
for x, y in kf:
    train.append(x)
    test.append(y)

In [131]:
len(train)

5

In [132]:
model_RF.fit(X[train[0]], cur_df.direction[train[0]])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=-2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [135]:
y = model_RF.predict(X[test[0]])

In [138]:
y.shape

(2391,)

In [139]:
cur_df.direction[test[0]].shape

(2391,)

In [141]:
score = model_RF.score(X[test[0]], cur_df.direction[test[0]])

In [142]:
score

0.7599330823923045

In [123]:
cross_v(X, cur_df)

Fitting model  RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=-2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Predicting


/usr/local/lib/python3.4/dist-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Number of features of the model must  match the input. Model n_features is 14670 and  input n_features is 2391 